In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
my_phone_number = '+5511991982436'

In [3]:
wd = webdriver.Chrome()

In [4]:
wd.implicitly_wait(10)

In [5]:
wd.get("https://amazon.com")

In [6]:
import time
time.sleep(5)

wd.find_element(By.ID, "nav-global-location-popover-link").click()
postcode_field = wd.find_element(By.ID, "GLUXZipUpdateInput")
postcode_field.send_keys("80021")
time.sleep(2)
wd.find_element(By.ID, "GLUXZipUpdate").click()
close_element = wd.find_element(By.XPATH, '//*[@id="a-popover-1"]/div/div[2]/span/span/span/button')
close_element.click()

In [7]:
wd.get("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(wd.page_source)

In [9]:
results = soup.findAll("div", {"data-component-type": "s-search-result"})
for result in results:
    title = result.find("span", {"class": "a-text-normal"})
    sponsored = result.find("span", {"class": "a-color-base"}, string="Sponsored")
    price = result.find("span", {"class": "a-price-whole"})
    if not sponsored and price:
        print(title.text)
        print(price.text)
        

Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4.0 Graphics Card Titanium and Black (Renewed)
717.
EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12G-P5-3967-KR, 12GB GDDR6X, iCX3 Technology, ARGB LED, Metal Backplate (Renewed)
699.
MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G OC - 12GB GDDR6X Graphic Card for PC Gaming, 320-Bit HDMI/DP, NVIDIA GPU, Tri-Frozr 2 Cooling, Ampere Architecture, Computer Video Graphics Card (Renewed)
684.
ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC Edition Graphics Card (PCIe 4.0, 12GB GDDR6X, HDMI 2.1, DisplayPort 1.4a, Dual Ball Fan Bearings, Military-Grade Certification, GPU Tweak II) (Renewed)
749.
MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 12G Computer Gaming Graphics Card 12GB GDRR6X 320-Bit HDMI/DP Nvlink Torx Fan 3 Ampere Architecture OC, NVIDIA GPU Video Cards for Gaming PC (Renewed)
689.
Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4.0 Graphics Card Titanium and Black
979.
EVGA GeForce RTX 3080 Ti XC3 Ultra Gaming, 12G-P5-3955-KR, 12GB GDDR6X, iCX3 C

In [10]:
def getPandaDfForSearchResults(searchResultsPage):
    rows=[]
    for result in searchResultsPage:
        title = result.find("span", {"class": "a-text-normal"})
        sponsored = result.find("span", {"class": "a-color-base"}, string="Sponsored")
        price = result.find("span", {"class": "a-price-whole"})
        url = result.find("a", {"class": "a-link-normal"})
        if price:
            row = [title.text, bool(sponsored), price.text, "https://amazon.com/" + url['href']]
            rows.append(row)
    
    df=pd.DataFrame.from_records(rows, columns=["Title", "Sponsored", "Price", "URL"])
    return df

In [11]:
import pandas as pd

nextPageCssClasses=[]
nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")


df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
while not "s-pagination-disabled" in nextPageCssClasses:
    soup = BeautifulSoup(wd.page_source)
    searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
    df = getPandaDfForSearchResults(searchResultsPage)
    df_all_search_results = pd.concat([df_all_search_results, df])
                              
    # go to next page
    nextPageLink.click()
    nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")
    nextPageCssClasses = nextPageLink.get_attribute("class").split()
    time.sleep(2)

if "s-pagination-disabled" in nextPageCssClasses:
    print("Reached last page of search results")

df_all_search_results    
                            

Reached last page of search results


,Title,Sponsored,Price,URL
0,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,717.,https://amazon.com//Geforce-Express-Graphics-T...
1,"EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12...",False,699.,https://amazon.com//EVGA-GeForce-12G-P5-3967-K...
2,MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G O...,False,684.,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,749.,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 1...,False,689.,https://amazon.com//MSI-GeForce-320-Bit-Archit...
...,...,...,...,...
12,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",False,"1,475.",https://amazon.com//EVGA-GeForce-Technology-Ba...
13,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,False,879.,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
14,MSI GeForce RTX 3090 Ti SUPRIM X 24G Gaming Gr...,False,"1,699.",https://amazon.com//GeForce-3090-SUPRIM-NVIDIA...
15,MSI GeForce RTX 4070 Ti Ventus 2X 12G OC,False,878.,https://amazon.com//MSI-GeForce-RTX-4070-Ventu...


In [12]:
df_all_search_results[df_all_search_results.Title.str.contains("3080 Ti")]

,Title,Sponsored,Price,URL
0,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,717.,https://amazon.com//Geforce-Express-Graphics-T...
1,"EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12...",False,699.,https://amazon.com//EVGA-GeForce-12G-P5-3967-K...
2,MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G O...,False,684.,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,749.,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 1...,False,689.,https://amazon.com//MSI-GeForce-320-Bit-Archit...
5,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,979.,https://amazon.com//Geforce-GDDR6X-Express-Gra...
6,"EVGA GeForce RTX 3080 Ti XC3 Ultra Gaming, 12G...",False,799.,https://amazon.com//EVGA-GeForce-12G-P5-3955-K...
7,MSI GeForce RTX 3080 Ti Gaming X Trio 12G Gami...,False,"1,329.",https://amazon.com//RTX-3080-Gaming-Trio-12GB/...
8,Gigabyte GeForce RTX 3080 Ti Gaming OC Graphic...,False,"1,229.",https://amazon.com//Gigabyte-GeForce-3080-GAMI...
9,MSI Gaming GeForce RTX 3080 Ti 12GB GDRR6X 320...,False,915.,https://amazon.com//MSI-GeForce-RTX-3080-12G/d...


In [13]:
df_all_search_results['Price'] = df_all_search_results['Price'].str.replace(",","")

In [14]:
df_all_search_results['Price'] = df_all_search_results['Price'].astype(float)

In [15]:
df_all_search_results

,Title,Sponsored,Price,URL
0,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,717.0,https://amazon.com//Geforce-Express-Graphics-T...
1,"EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12...",False,699.0,https://amazon.com//EVGA-GeForce-12G-P5-3967-K...
2,MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G O...,False,684.0,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,749.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 1...,False,689.0,https://amazon.com//MSI-GeForce-320-Bit-Archit...
...,...,...,...,...
12,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",False,1475.0,https://amazon.com//EVGA-GeForce-Technology-Ba...
13,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,False,879.0,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
14,MSI GeForce RTX 3090 Ti SUPRIM X 24G Gaming Gr...,False,1699.0,https://amazon.com//GeForce-3090-SUPRIM-NVIDIA...
15,MSI GeForce RTX 4070 Ti Ventus 2X 12G OC,False,878.0,https://amazon.com//MSI-GeForce-RTX-4070-Ventu...


In [16]:
df_all_search_results = df_all_search_results[(df_all_search_results.Title.str.contains("3080 Ti")) & (df_all_search_results.Price <= 1500)]

In [17]:
df_all_search_results

,Title,Sponsored,Price,URL
0,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,717.0,https://amazon.com//Geforce-Express-Graphics-T...
1,"EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12...",False,699.0,https://amazon.com//EVGA-GeForce-12G-P5-3967-K...
2,MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G O...,False,684.0,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,749.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 1...,False,689.0,https://amazon.com//MSI-GeForce-320-Bit-Archit...
5,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,979.0,https://amazon.com//Geforce-GDDR6X-Express-Gra...
6,"EVGA GeForce RTX 3080 Ti XC3 Ultra Gaming, 12G...",False,799.0,https://amazon.com//EVGA-GeForce-12G-P5-3955-K...
7,MSI GeForce RTX 3080 Ti Gaming X Trio 12G Gami...,False,1329.0,https://amazon.com//RTX-3080-Gaming-Trio-12GB/...
8,Gigabyte GeForce RTX 3080 Ti Gaming OC Graphic...,False,1229.0,https://amazon.com//Gigabyte-GeForce-3080-GAMI...
9,MSI Gaming GeForce RTX 3080 Ti 12GB GDRR6X 320...,False,915.0,https://amazon.com//MSI-GeForce-RTX-3080-12G/d...


In [18]:
import pandas as pd



def getSearchResultsForPage(url):
    wd.get(url)
    nextPageCssClasses=[]
    nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")


    df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
    while not "s-pagination-disabled" in nextPageCssClasses:    
        soup = BeautifulSoup(wd.page_source)
        searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
        df = getPandaDfForSearchResults(searchResultsPage)
        df_all_search_results = pd.concat([df_all_search_results, df])

        # go to next page
        nextPageLink.click()
        nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")
        nextPageCssClasses = nextPageLink.get_attribute("class").split()
        time.sleep(2)

    if "s-pagination-disabled" in nextPageCssClasses:
        print("Reached last page of search results")

    df_all_search_results['Price'] = df_all_search_results['Price'].str.replace(",","")
    df_all_search_results['Price'] = df_all_search_results['Price'].astype(float)
    return df_all_search_results    
                            

In [19]:
getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

Reached last page of search results


,Title,Sponsored,Price,URL
0,Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4....,False,717.0,https://amazon.com//Geforce-Express-Graphics-T...
1,"EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12...",False,699.0,https://amazon.com//EVGA-GeForce-12G-P5-3967-K...
2,MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G O...,False,684.0,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,749.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 1...,False,689.0,https://amazon.com//MSI-GeForce-320-Bit-Archit...
...,...,...,...,...
12,"EVGA GeForce RTX 3090 FTW3 Ultra Gaming, 24GB ...",False,1475.0,https://amazon.com//EVGA-GeForce-Technology-Ba...
13,MSI Gaming GeForce RTX 3090 24GB GDRR6X 384-Bi...,False,879.0,https://amazon.com//MSI-GeForce-Tri-Frozr-Arch...
14,MSI GeForce RTX 3090 Ti SUPRIM X 24G Gaming Gr...,False,1699.0,https://amazon.com//GeForce-3090-SUPRIM-NVIDIA...
15,MSI GeForce RTX 4070 Ti Ventus 2X 12G OC,False,878.0,https://amazon.com//MSI-GeForce-RTX-4070-Ventu...


In [20]:
import pywhatkit as kit
def sendWhatsAppMessage(df):
    for index, graphicsCardRow in df.iterrows():
        print(graphicsCardRow.URL)
        wd.get(graphicsCardRow.URL)
        screenshotFilepath="C:\\Users\\armin\\Downloads\\screenshot_{index}.png"
        wd.save_screenshot(screenshotFilepath)
        kit.sendwhats_image(my_phone_number, screenshotFilepath, graphicsCardRow.URL, tab_close=True)
       
        
        

    

In [21]:
while (True):
    df = getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")
    
    df_query = df[(df.Title.str.contains("3080 Ti")) & (df.Price <= 2500)]
    
    if (not df_query.empty):
        print("Found products that I want to buy")
        sendWhatsAppMessage(df_query)
        print(df_query)
        break
    else:
        time.sleep(60)
    

Reached last page of search results
Found products that I want to buy
https://amazon.com//Geforce-Express-Graphics-Titanium-Renewed/dp/B09WSC85GP/ref=sr_1_1?dib=eyJ2IjoiMSJ9.nVzd2nIdxZ8mGjehwv9iLvrZzX7qpwlY2_aZpK0g6hVbLilvVKmn999IwvJS8TiALf5ORyweAhb0GZ7cttPHbb1WvY2YaxP0JhwNyjv59ZImJaLGGN_0tPhlISbgQqDVRv9VZs0mGERlAgkKxEImbeKStPTnrAqDjPjBJrJSEu8cjiBVe9OKQor_B4Q9kGPwQMpjiTrm-USUBEXmeokQdwVqHyJ9uUO_Tyv8dmaX3qhMp6A3bSgXq3bovzVRBPtSJG5XkSI0TPp61AmcfuEiIGbjU-lDDl_IHXIkUlDLUFg.5jWZwCfZJbzfMWJuplvoGeNawOVFxnp9jORmgdYO6G0&dib_tag=se&keywords=rtx+3080+ti&qid=1717009082&refinements=p_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&rnid=2661599011&s=pc&sr=1-1


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\armin\\Downloads\\screenshot_{index}.png'

In [ ]:
import pywhatkit as kit